This notebook explores a number of different potentials of the Milky Way to determine how much they may influence an orbit.

In [ ]:
using CairoMakie
using Revise
using LilGuys, Arya

In [ ]:
# use natural units (G=1) for everything. if the potential uses different units
# we are best off to change the inputs rather than Agama's units
using PythonCall
agama = pyimport("agama")

## Setup
Here I use Sculptor.

In [ ]:
include("agama_utils.jl")

In [ ]:
obs_props_filename = ENV["DWARFS_ROOT"] * "/observations/sculptor/observed_properties.toml"

In [ ]:
# sculptor's adpoted mean orbit for this project
icrs = LilGuys.coord_from_file(obs_props_filename)
icrs_err = LilGuys.coord_err_from_file(obs_props_filename)

In [ ]:
gc = LilGuys.transform(Galactocentric, icrs)

In [ ]:
ep20 = load_agama_potential("EP2020.ini")


In [ ]:
orbit = calc_orbit(gc, ep20, units=:code, time=-10/T2GYR)

In [ ]:
plot_r_t(orbit)

In [ ]:
plot_y_z(orbit)

# Potentials
See each file for a definition of the parameters and attributes.
Note that Agama does combine potentials together so they are not simply a list of their components.

In [ ]:
mm11 = load_agama_potential("mcmillan11.ini")
mm17 = load_agama_potential("mcmillan17.ini")
bt08 = load_agama_potential("BT08.ini")
piffl14 = load_agama_potential("piffl_14.ini")
J95 = load_agama_potential("J95.ini")
galpy14 = load_agama_potential("galpy_2014.ini")
pouliasis17 = load_agama_potential("pouliasis+2017.ini")
ablimit20 = load_agama_potential("ablimit+2020.ini")
nitschai20 = load_agama_potential("nitschai+2020.ini")
pouliasis17b = load_agama_potential("pouliasis+2017b.ini")
AS91 = load_agama_potential("AS1991.ini")

In [ ]:
potentials_mcmillan = [
    "EP20" => ep20,
    "mcmillan 11" => mm11,
    "mcmillan 17" => mm17,
    ]

In [ ]:
potentials_light = [
    "EP20" => ep20,
    "bt08" => bt08,
    "galpy14" => galpy14,
    "AS1991" => AS91,
    "ablimit + 2020" => ablimit20,
    ]


In [ ]:
potentials_heavy = [
    "EP20" => ep20,
    "piffl14" => piffl14,
    "J95" => J95,
    "nitschai+20" => nitschai20,
    "pouliasis+17" => pouliasis17,
    "pouliasis+17b" => pouliasis17,
]

# Potential comparisons

In [ ]:
function compare_v_circ(potentials)
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel = "log r / kpc",
        ylabel = L"$v_\textrm{circ}$ / km\,s$^{-1}$",
    )

    for (label, pot) in potentials
        plot_v_circ!(pot, label=label)
    end

    axislegend(position=:rb)
    fig
end

In [ ]:
function plot_all(gc, pot)
    orbit = calc_orbit(gc, pot)
    plot_r_t(orbit) |> display
    plot_y_z(orbit) |> display
end

In [ ]:
function plot_all(gc, pots)

    
    orbits = [calc_orbit(gc, pot, units=:code, time=-10 / T2GYR) for (name, pot) in pots]

    display(compare_v_circ(pots))
    
    fig = Figure()
    ax = axis_r_t(fig[1, 1])
    
    for i in 1:length(pots)
        plot_r_t!(ax, orbits[i], label=pots[i].first)
    end

    axislegend()
    
    display(fig)


    fig = Figure()
    ax = axis_y_z(fig[1,1])


    for i in 1:length(pots)
        plot_y_z!(ax, orbits[i], label=pots[i].first)
    end

    axislegend()
    display(fig)
end

In [ ]:
plot_all(gc, potentials_mcmillan)

In [ ]:
plot_all(gc, potentials_light)

In [ ]:
plot_all(gc, potentials_heavy)